<a href="https://colab.research.google.com/github/thatninja13/openai-quickstart-node/blob/master/Simplified_CC12M.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**The MOST Powerful Art Generator you need to learn RIGHT NOW!!!**

*Art Generator has been upgraded and we can now generate art 10 times faster with lower steps!!!*

#**1)** Download everything required for Art Generator ❗❗❗

In [ ]:
# @title Licensed under the MIT License

# Copyright (c) 2022 Katherine Crowson

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.

In [ ]:
# Check the GPU

!mkdir steps
!nvidia-smi

### Install dependencies (no need to rerun this section if you restart the notebook runtime)

In [ ]:
# Install dependencies

!pip install ftfy regex requests tqdm
!git clone --recursive https://github.com/crowsonkb/v-diffusion-pytorch

In [ ]:
# Download the diffusion model
# SHA-256: 4fc95ee1b3205a3f7422a07746383776e1dbc367eaf06a5b658ad351e77b7bda

!mkdir v-diffusion-pytorch/checkpoints
!curl -L https://v-diffusion.s3.us-west-2.amazonaws.com/cc12m_1_cfg.pth > v-diffusion-pytorch/checkpoints/cc12m_1_cfg.pth

### Import modules and load models

In [ ]:
# Imports

import gc
import math
import sys

from IPython import display
import torch
from torchvision import utils as tv_utils
from torchvision.transforms import functional as TF
from tqdm.notebook import trange, tqdm

sys.path.append('/content/v-diffusion-pytorch')

from CLIP import clip
from diffusion import get_model, sampling, utils

In [ ]:
# Load the models

model = get_model('cc12m_1_cfg')()
_, side_y, side_x = model.shape
model.load_state_dict(torch.load('v-diffusion-pytorch/checkpoints/cc12m_1_cfg.pth', map_location='cpu'))
model = model.half().cuda().eval().requires_grad_(False)
clip_model = clip.load(model.clip_model, jit=False, device='cpu')[0]

In [ ]:
# side_y = 512
# side_x = 512

# **2)** Parameters for The Art Generator 🤖


***-Only these heights and widths work: 4,8,16,32,64,128,256,512***

-Change the seed if you want different results!

In [ ]:
#@title Settings
import random
prompt = 'dark story of star wars comic'  #@param {type:"string"}

height =  256#@param {type:"integer"}
width =  256#@param {type:"integer"}
side_x = width
side_y = height
#@markdown Specify the number of diffusion timesteps (default is 50, can lower for faster but lower quality sampling).
steps =   50#@param {type:"integer"}


#@markdown Sample this many images.
n_images =   8#@param {type:"integer"}

weight = 5 

#@markdown Set to 0 for deterministic (DDIM) sampling, 1 (the default) for stochastic (DDPM) sampling, and in between to interpolate between the two. 0 is preferred for low numbers of timesteps.
eta =   0#@param {type:"number"}

#@markdown The random seed. Change this to sample different images. -1 means completely random seed!
seed =   -1#@param {type:"integer"}

#@markdown Display progress every this many timesteps.
display_every = 5  #@param {type:"integer"}

save_progress_video = True #@param {type:"boolean"}


# **3)** Generate Art 😎


In [ ]:
#@title Run the Generator
from PIL import ImageFile, Image
import numpy as np
import os
target_embed = clip_model.encode_text(clip.tokenize(prompt)).float().cuda()
from IPython.display import clear_output

def cfg_model_fn(x, t):
    """The CFG wrapper function."""
    n = x.shape[0]
    x_in = x.repeat([2, 1, 1, 1])
    t_in = t.repeat([2])
    clip_embed_repeat = target_embed.repeat([n, 1])
    clip_embed_in = torch.cat([torch.zeros_like(clip_embed_repeat), clip_embed_repeat])
    v_uncond, v_cond = model(x_in, t_in, clip_embed_in).chunk(2, dim=0)
    v = v_uncond + (v_cond - v_uncond) * weight
    return v
save_name = 0.00000000

def display_callback(info):
    global save_name
    save_name += 0.00000001
    nrow = math.ceil(info['pred'].shape[0]**0.5)
    grid = tv_utils.make_grid(info['pred'], nrow, padding=0)
    utils.to_pil_image(grid).save(f"./steps/%.8f.png" % save_name)
    
    if info['i'] % display_every == 0:
        nrow = math.ceil(info['pred'].shape[0]**0.5)
        grid = tv_utils.make_grid(info['pred'], nrow, padding=0)
        tqdm.write(f'Step {info["i"]} of {steps}:')
        display.display(utils.to_pil_image(grid))
        # utils.to_pil_image(grid).save("./steps/Test.png")
        
        tqdm.write(f'')


def run(seed):
    print("Prompt is: " + prompt)
    
    if seed == -1:
       seed = random.randint(0, 2**32)
      # print(seed)
    print("Seed is: " + str(seed))
    gc.collect()
    torch.cuda.empty_cache()
    torch.manual_seed(seed)
    x = torch.randn([n_images, 3, side_y, side_x], device='cuda')
    t = torch.linspace(1, 0, steps + 1, device='cuda')[:-1]
    step_list = utils.get_spliced_ddpm_cosine_schedule(t)
    outs = sampling.sample(cfg_model_fn, x, step_list, eta, {}, callback=display_callback)
    tqdm.write('Done!')
    for i, out in enumerate(outs):
        filename = f'out_{i}.png'
        utils.to_pil_image(out).save(filename)
        display.display(display.Image(filename))


run(seed)
frames = []
files = []
init_frame = 1 #Este es el frame donde el vídeo empezará
last_frame = steps #Puedes cambiar i a el número del último frame que quieres generar. It will raise an error if that number of frames does not exist.

directory = 'steps'
    # iterate over files in
    # that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
        # checking if it is a file
    files.append(f)
for i in range(init_frame,last_frame): #
    # save_name2 += 0.00000001
    # filename = f"steps/{save_name2}.png"
    # print(files[i])
    frames.append(Image.open(files[i]))
frames[-1].save("finalgrid.png")


if save_progress_video:

    init_frame = 1 #Este es el frame donde el vídeo empezará
    last_frame = steps #Puedes cambiar i a el número del último frame que quieres generar. It will raise an error if that number of frames does not exist.

    min_fps = 10
    max_fps = 30

    total_frames = last_frame-init_frame

    length = 15 #Tiempo deseado del vídeo en segundos

    # import required module
    import os
    # assign directory
    directory = 'steps'
    
    files = []

    # iterate over files in
    # that directory
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        # checking if it is a file
        files.append(f)


    files.sort()
    # print(files)


    frames = []
    save_name2 = 0.00000000
    tqdm.write('Generating video...')
    for i in range(init_frame,last_frame): #
        # save_name2 += 0.00000001
        # filename = f"steps/{save_name2}.png"
        # print(files[i])
        frames.append(Image.open(files[i]))
    # frames[-1].save("finalgrid.png")
    #fps = last_frame/10
    fps = np.clip(total_frames/length,min_fps,max_fps)

    from subprocess import Popen, PIPE
    p = Popen(['ffmpeg', '-y', '-f', 'image2pipe', '-vcodec', 'png', '-r', str(fps), '-i', '-', '-vcodec', 'libx264', '-r', str(fps), '-pix_fmt', 'yuv420p', '-crf', '17', '-preset', 'veryslow', 'video.mp4'], stdin=PIPE)
    for im in tqdm(frames):
        im.save(p.stdin, 'PNG')
    p.stdin.close()

    p.wait()
    print("Video finished Rendering")

In [ ]:
#@markdown Show Generated Video in Colab
from base64 import b64encode
mp4 = open('video.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
display.HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

#🌟🌟🌟 **Beautiful Prompts you should try!** 🌟🌟🌟

More prompts added everyday at: https://www.reddit.com/r/TheGenerat0rs/


    The Tower Of Illusions in the clouds by Thomas Kinkade
    The Tower of Illusions in the clouds by Thomas Kinkade by Lisa Frank
    dark story of star wars comic
    The Tower of Illusions in the clouds by Thomas Kinkade WPAP
    Starry Night in the style of Lisa Frank
    a beautiful epic fantasy of a 🤖
    a futuristic boat
    fantasy world in unreal engine
    retrowave castle city by Tyler Edlin in the style of John Constable
    a new algorithm
    lighthouse on the edge of tomorrow, surreal, digital art
    futuristic Eiffel Tower, dslr
    concept lanscape environment fantasy
    voxel minecraft house
    Architectural Visualizations in unreal engine
    Architectural Concepts spaceship artstation hd
    ahri fan art artstation hd
    bmw automotive digital 3D in unreal engine 4K 
    concept art fan art artstation hd
    the infinity realm concept art digital 2d illustration
    hyperrealistic psychedelic ophanim

-
#**Want to Super-Enhance the Generated Image with another advanced A.I.? Enhances a 500x500 image to 1760x1760 with no quality loss!!!**: 

-

https://colab.research.google.com/drive/1O8kr-iyRll5eQsaOH-vQIc3ImEKWsvF-?usp=sharing